In [1]:
from tqdm import tqdm
import re
import numpy as np
from collections import defaultdict
import heapq as heap

import itertools

with open('data.txt') as file:
    data = file.read().splitlines()

In [2]:
valves = {}

for valve in data:
    v = {}
    a,b = valve.split(';')

    v['open'] = False

    name = a.split(' ')[1]
    v['rate'] = int(a.split(' ')[-1].split('=')[-1])
    
    nexts = b.split(', ')
    nexts[0] = nexts[0].split(' ')[-1]

    v['next'] = nexts
    v['name'] = name
    
    valves[name] = v

In [4]:
cache = {}


def get_best_path(key, open_valves, minutes_left, closed_valves):

    cache_key = (key, tuple(set(open_valves)), minutes_left)
    if cached := cache.get(cache_key, None):
        return cached

    alts = []

    if minutes_left == 0 or len(valves.items()) == len(open_valves):
        return 0

    if key not in open_valves:

        new_open_vales = open_valves.copy()
        new_open_vales.add(key)

        new_closed = closed_valves.copy()
        new_closed.remove(key)

        release = (minutes_left - 1) * valves[key]["rate"] + get_best_path(
            key,
            new_open_vales,
            minutes_left=minutes_left - 1,
            closed_valves=new_closed,
        )
        alts.append(release)

    for valve in valves[key]["next"]:
        release = get_best_path(
            valve,
            open_valves,
            minutes_left=minutes_left - 1,
            closed_valves=closed_valves,
        )
        alts.append(release)

    total_score = max(alts)
    cache[cache_key] = total_score
    return total_score


zero_rates = [key for key, val in valves.items() if val["rate"] == 0]
valves_left = [key for key, val in valves.items() if key not in zero_rates]

get_best_path("AA", set(zero_rates), 30, set(valves_left))


1653

In [5]:
full = set(valves_left)

combs = []

for n in range(len(full)):
    my = list(itertools.combinations(full, n))
    elephant = [[a for a in full if a not in b] for b in my]
    combs += list(zip(my, elephant))


In [7]:
scores = []

for c in tqdm(combs):
    my, elephant = c
    my_path = get_best_path("AA", set(zero_rates + list(elephant)), 26, set(my))
    elephant_path = get_best_path("AA", set(zero_rates + list(my)), 26, set(elephant))
    scores.append(my_path + elephant_path)

max(scores)

100%|██████████| 32767/32767 [15:29<00:00, 35.26it/s]   


2223